In [1]:
import pandas as pd
#make initial dataframe from songList

Songlist_file = "Metadata-ComposerSongNames.txt"

with open(Songlist_file, encoding = "utf-8") as f:
    songList = f.readlines()

kr_songList = []
kr_composerList = []
en_songList = []
en_composerList = []
for item in songList:
    item = item.split("//")
    kr_songList.append(item[0])
    kr_composerList.append(item[1])
    en_songList.append(item[2])
    en_composerList.append(item[3][:-1])

    
songList_df = pd.DataFrame({'song':en_songList, 'composer':en_composerList , 'kr_song':kr_songList, 'kr_composer':kr_composerList})

#songList_df.to_csv("songList.csv", index = False)


songList_df


,song,composer,kr_song,kr_composer
0,STAGER,Ras,스테이저,Ras
1,Little Munchkin,Castellia,리틀 먼치킨,Castellia
2,Sudden Appearance Image,Blacklolita,서든 어페어런스 이미지,Blacklolita
3,TOMBOY,(G)I-DLE,톰보이,(여자)아이들
4,"Simon Says, EURODANCE!! (feat. Sara☆M)",Jehezukiel☆KAZEMURA☆Hideky,"사이먼 세이즈, 유로댄스!! (feat. Sara☆M)",Jehezukiel☆KAZEMURA☆Hideky
...,...,...,...,...
539,Close Your Eye,BanYa,눈을 감아,반야
540,Ugly duck Toccata,Banya Production,미운 오리 토카타,반야 프로덕션
541,Set Up Me2 Mix,Banya Production,셋업 미투 믹스,반야 프로덕션
542,Unique - SHORT CUT -,SHK,유니크 - SHORT CUT -,SHK


In [2]:
import json

en_composers_list = []
kr_composers_list = []
en_songs_list = []
kr_songs_list = []
for i in range(len(songList_df)):
    if en_composerList[i] not in en_composers_list:
        en_composers_list.append(en_composerList[i])
        kr_composers_list.append(kr_composerList[i])
        en_songs_list.append([en_songList[i]])
        kr_songs_list.append([kr_songList[i]])
    else:
        idx = en_composers_list.index(en_composerList[i])
        en_songs_list[idx].append(en_songList[i])
        kr_songs_list[idx].append(kr_songList[i])

composerList_df = pd.DataFrame({'composer:': en_composers_list, 'kr_composer': kr_composers_list, 'songs': en_songs_list, 'kr_songs': kr_songs_list})

#composerList_df.to_csv("composerList.csv", index = False)



In [3]:
#Find songs by composer

import ast
#returns set of composers by song
def find_list_of_songs_by_composer_raw(composer_csv, composer_name):
    composerlist_selected = composer_csv[composer_csv['composer'].str.contains(composer_name, case = False)]
    songlist = []
    for i in range(len(composerlist_selected)):
        songlist_raw = composerlist_selected['songs'].values[i]
        real_songlist = ast.literal_eval(songlist_raw)
        songlist.append(real_songlist)
    flattened_songlist = [item for sublist in songlist for item in sublist]
    return set(flattened_songlist)

#returns set of songs by composer
def find_composer_by_song(composer_csv, song_name):
    composerlist_selected = composer_csv[composer_csv['songs'].str.contains(song_name, case = False)]
    composerlist = []
    for i in range(len(composerlist_selected)):
        composerlist.append(composerlist_selected['composer'].values[i])
    return set(composerlist)

#returns set of songs by composer with ad hoc
def find_list_of_songs_by_composer(composer_csv, composer_name):
    if composer_name.lower() == "max":
        setA = find_list_of_songs_by_composer_raw(composer_csv, "MAX")
        setB = find_list_of_songs_by_composer_raw(composer_csv, "SLAM")
        return setA.union(setB)
    if composer_name.lower() == "kien":
        setA = find_list_of_songs_by_composer_raw(composer_csv, "kien")
        setB = find_list_of_songs_by_composer_raw(composer_csv, "STREME REVERIE")
        return setA.union(setB)
    if "lunatic" in composer_name.lower() or "cosmo" in composer_name.lower():
        setA = find_list_of_songs_by_composer_raw(composer_csv, "Lunatic Sounds")
        setB = find_list_of_songs_by_composer_raw(composer_csv, "Cosmograph")
        setC = find_list_of_songs_by_composer_raw(composer_csv, "houseplan")
        return setA.union(setB).union(setC)
    if composer_name.lower() == "banya":
        setA = find_list_of_songs_by_composer_raw(composer_csv, "Banya")
        setB = find_list_of_songs_by_composer_raw(composer_csv, "Banya Production")
        return setA - setB
    if composer_name.lower() == "xi":
        setA = find_list_of_songs_by_composer_raw(composer_csv, "xi")
        setB = find_list_of_songs_by_composer_raw(composer_csv, "Sanxion7")
        return setA - setB
    if composer_name.lower() == "quree" or "wyve" in composer_name.lower():
        setA = find_list_of_songs_by_composer_raw(composer_csv, composer_name)
        setB = find_list_of_songs_by_composer_raw(composer_csv, "Qwertism")
        return setA.union(setB)
    return find_list_of_songs_by_composer_raw(composer_csv, composer_name)



composer_csv = pd.read_csv("composerList.csv")
print(find_list_of_songs_by_composer(composer_csv, "pazolite"))
print(find_composer_by_song(composer_csv, "cross"))


{'Curiosity Overdrive', 'Slapstick Parfait', 'Crossing Delta'}
{'HyuN feat. Syepias', 't+pazolite+Ginkiha+Hommarju', 'HyuN', 'Brandy'}


In [4]:
songList_df_new = pd.read_csv("songList.csv")
def find_korean_song_name(songlist, songname):
    anslist = songlist[songlist['song'].str.contains(songname, case = False)]['kr_song']
    if len(anslist) == 0:
        return "Could not find any songs"
    else:
        return set(anslist.values)

def find_korean_song_name_exact(songlist, songname):
    ans = songlist[songlist['song'] == songname]['kr_song']
    if len(ans) == 0:
        print(f"case: {songname} // Could not find any songs")
        return "N/A"
    return songlist[songlist['song'] == songname]['kr_song'].values[0]



find_korean_song_name(songList_df_new, "prim")

find_korean_song_name_exact(songList_df_new, "Destr0er")


case: Destr0er // Could not find any songs


'N/A'

In [5]:
composer_csv = pd.read_csv("composerList.csv")
kr_songs = []
for i in range(len(composer_csv)):
    add_composers = []
    #print(composer_csv['songs'].values[i])
    real_songlist = ast.literal_eval(composer_csv['songs'].values[i])
    #print(len(real_songlist))
    for j in range(len(real_songlist)):
        add_composers.append(find_korean_song_name_exact(songList_df_new, real_songlist[j]))
    #kr_songs.append(find_list_of_songs_by_composer(composer_csv, composer_csv['songs'].values[i]))
    kr_songs.append(add_composers)
kr_songs
composer_csv['kr_songs'] = kr_songs
composer_csv

composer_csv.to_csv("composerList_new.csv", index = False)

In [6]:
composer_csv = pd.read_csv("composerList_new.csv")
composer_csv

def find_kr_composer_name(composer_csv, composer_name):
    return composer_csv[composer_csv['composer'].str.contains(composer_name, case = False)]['kr_composer'].values[0]

find_kr_composer_name(composer_csv, "sanxion")

'Sanxion7'

In [15]:
#songList_df_new["kr_song_proxy"] = songList_df_new["kr_song"]

#songList_df_new.to_csv("songList_writeProxies.csv", index = False)
songList_df_new2 = pd.read_csv("songList_writeProxies.csv")


In [16]:
def is_short_cut(songname):
    if "short cut" in songname.lower():
        return True
    return False

def is_full_song(songname):
    if "full song" in songname.lower():
        return True
    return False

def get_full_song_name(songname):
    return songname[:-14]

def get_short_cut_name(songname):
    return songname[:-14]
import json
arcade_metadata = json.load(open("arcadeSong_metadata.json"))

arcade_metadata[1]
indexlist = []
bpmlist = []
real_bpmlist = []
versionlist = []
count = 0
songtypelist = []

for i in range(len(songList_df_new2)):
    songname = songList_df_new2['song'].values[i]
    #print(songname, end = " ")
    isMatched = False
    for j in range(len(arcade_metadata)):
        if songname == arcade_metadata[j]['song']:
            #print("is matched")
            indexlist.append(arcade_metadata[j]['index'])
            bpmlist.append(arcade_metadata[j]['bpm'])
            real_bpmlist.append(arcade_metadata[j]['bpm'])
            versionlist.append(arcade_metadata[j]['version'])
            songtypelist.append('Arcade')
            count += 1
            isMatched = True
    if not isMatched:
        isRemix = True
        if is_full_song(songname):
            isRemix = False
            fullsongname = get_full_song_name(songname)
            songtypelist.append('Full Song')
            for j in range(len(arcade_metadata)):
                if fullsongname == arcade_metadata[j]['song']:
                    indexlist.append("F" + arcade_metadata[j]['index'][1:])
                    bpmlist.append(arcade_metadata[j]['bpm'])
                    real_bpmlist.append(arcade_metadata[j]['bpm'])
                    versionlist.append("N/A")
                    count += 1
                    isMatched = True
            if isMatched == False:
                indexlist.append("N/A")
                bpmlist.append("N/A")
                real_bpmlist.append("N/A")
                versionlist.append("N/A")
                count += 1
        if is_short_cut(songname):
            isRemix = False
            shortsongname = get_short_cut_name(songname)
            songtypelist.append('Short Cut')
            for j in range(len(arcade_metadata)):
                if shortsongname == arcade_metadata[j]['song']:
                    indexlist.append("S" + arcade_metadata[j]['index'][1:])
                    bpmlist.append(arcade_metadata[j]['bpm'])
                    real_bpmlist.append(arcade_metadata[j]['bpm'])
                    versionlist.append("N/A")
                    count += 1
                    isMatched = True
            if isMatched == False:
                indexlist.append("N/A")
                bpmlist.append("N/A")
                real_bpmlist.append("N/A")
                versionlist.append("N/A")
                count += 1
        if isRemix == True:
            songtypelist.append('Remix')
            indexlist.append("N/A")
            bpmlist.append("N/A")
            real_bpmlist.append("N/A")
            versionlist.append("N/A")
            count += 1
        continue



print(count)
print(len(arcade_metadata))
print(len(real_bpmlist))
for i in range(len(arcade_metadata)):
    if arcade_metadata[i]['index'] not in indexlist:
        print(arcade_metadata[i]['song'])

549
444
549


In [18]:
songList_df_new2["index"] = indexlist
songList_df_new2["bpm"] = bpmlist
songList_df_new2["main_bpm"] = real_bpmlist
songList_df_new2["version"] = versionlist
songList_df_new2["song_type"] = songtypelist

songList_df_new2


,song,composer,kr_song,kr_song_proxy,index,bpm,main_bpm,version,song_type
0,STAGER,Ras,스테이저,스테이져,CS683,146,146,PHOENIX,Arcade
1,Little Munchkin,Castellia,리틀 먼치킨,리틀 먼치킨,CS360,180,180,PHOENIX,Arcade
2,Sudden Appearance Image,Blacklolita,서든 어페어런스 이미지,서든 어피어런스 이미지,CS362,170,170,PHOENIX,Arcade
3,TOMBOY,(G)I-DLE,톰보이,톰보이,CS823,124,124,PHOENIX,Arcade
4,"Simon Says, EURODANCE!! (feat. Sara☆M)",Jehezukiel☆KAZEMURA☆Hideky,"사이먼 세이즈, 유로댄스!! (feat. Sara☆M)",사이몬 세이즈,CS361,157,157,PHOENIX,Arcade
...,...,...,...,...,...,...,...,...,...
544,PRiMA MATERiA,xi,프ㄹi마 마테ㄹi아,프리마 마테리아,CS366,187,187,PHOENIX,Arcade
545,Solfeggietto,Fiverwater,솔페지에토,솔페지에토,CS364,157,157,PHOENIX,Arcade
546,Destr0yer,Sakuzyo,디스트로0ㅣ어,디스트로이어,CS684,90,90,PHOENIX,Arcade
547,Lucid Dream,Cosmograph,루시드 드림,루시드 드림,CS363,173,173,PHOENIX,Arcade


In [20]:
def df_stripper(df):
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
def find_kr_composer_name(composer_csv, composer_name):
    ans = composer_csv[composer_csv['composer'].str.contains(composer_name, case = False)]['kr_composer']
    if len(ans) == 0:
        print(f"case: {composer_name} // Could not find any composers")
        return "N/A"
    else:
        return composer_csv[composer_csv['composer'].str.contains(composer_name, case = False)]['kr_composer'].values[0]

kr_composername = []

songList_df_new2 = df_stripper(songList_df_new2)
composer_csv = df_stripper(composer_csv)

for i in range(len(songList_df_new2)):
    #print(i)
    #print(songList_df_new2['composer'].values[i])
    composer_name = songList_df_new2['composer'].values[i]
    kr_composername.append(find_kr_composer_name(composer_csv, composer_name))
songList_df_new2["kr_composer"] = kr_composername
songList_df_new2['song_type'] = songtypelist


C:\Users\chorj\AppData\Local\Temp\ipykernel_110816\1925321277.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ans = composer_csv[composer_csv['composer'].str.contains(composer_name, case = False)]['kr_composer']


case: (G)I-DLE // Could not find any composers
case: (G)I-DLE // Could not find any composers
case: t+pazolite // Could not find any composers
case: TANO*C Sound Team // Could not find any composers
case: A:llha (ANE + 2riA) // Could not find any composers
case: t+pazolite+Ginkiha+Hommarju // Could not find any composers
case: t+pazolite // Could not find any composers
case: MAX & Rorychesell(SID-SOUND) // Could not find any composers
case: MAX&Seorryang(SID-SOUND) // Could not find any composers


In [21]:
songList_df_new2[530:]

,song,composer,kr_song,kr_song_proxy,index,bpm,main_bpm,version,song_type,kr_composer
530,Exceed2 Opening - SHORT CUT -,Banya,익시드2 오프닝 - SHORT CUT -,익시드2 오프닝 - SHORT CUT -,N/A,N/A,N/A,N/A,Short Cut,반야
531,msgoon RMX pt.6,msgoon,엠.에스.군 리믹스 파트.6,엠에스군 리믹스 파트.6,N/A,N/A,N/A,N/A,Remix,엠.에스.군
532,Turkey Virus,Banya Production,터키 바이러스,터키 바이러스,N/A,N/A,N/A,N/A,Remix,반야 프로덕션
533,Chopsticks Challenge - FULL SONG -,YAHPP,젓가락 변주곡 - FULL SONG -,젓가락 변주곡 - FULL SONG -,FS094,128,128,N/A,Full Song,이 얍
534,K.O.A : Alice In Wonderworld - SHORT CUT -,Banya Production,케이.오.에이 : 엘리스 인 원더월드 - SHORT CUT -,케이.오.에이 : 엘리스 인 원더월드 - SHORT CUT -,SS091,156,156,N/A,Short Cut,반야 프로덕션
535,Final Audition 2 - SHORT CUT -,Banya,파이널 오디션 2 - SHORT CUT -,파이널 오디션 2 - SHORT CUT -,SS011,130,130,N/A,Short Cut,반야
536,B.P Classic Remix 2,Banya Production,반야 프로덕션 클래식 리믹스 2,반야 프로덕션 클래식 리믹스 2,N/A,N/A,N/A,N/A,Remix,반야 프로덕션
537,Take Out - SHORT CUT -,SHK,테이크 아웃 - SHORT CUT -,테이크 아웃 - SHORT CUT -,SS124,170,170,N/A,Short Cut,SHK
538,Pop The Track - SHORT CUT -,J-Mi & Midi-D & Ft. Hanna Stockzell,팝 더 트랙 - SHORT CUT -,팝 더 트랙 - SHORT CUT -,SS616,160,160,N/A,Short Cut,J-Mi & Midi-D & Ft. Hanna Stockzell
539,Close Your Eye,BanYa,눈을 감아,눈을 감아,CS007,104,104,1st - Zero,Arcade,반야


In [22]:
songList_rearrange =  songList_df_new2[['index','song', 'kr_song', 'kr_song_proxy', 'composer', 'kr_composer','bpm', 'main_bpm', 'version', 'song_type']]

In [23]:
songList_rearrange.to_csv("songList_tofill.csv", index = False)

In [26]:
songList_df_temp = songList_df_new2[['song', 'version', 'song_type']]


In [28]:
songList_df_temp

songList_df_temp.to_csv("songList_temp.csv", index = False)

In [29]:
songList_df_check = pd.read_csv("songList_temp.csv")

songList_df_check

,song,version,song_type
0,STAGER,PHOENIX,Arcade
1,Little Munchkin,PHOENIX,Arcade
2,Sudden Appearance Image,PHOENIX,Arcade
3,TOMBOY,PHOENIX,Arcade
4,"Simon Says, EURODANCE!! (feat. Sara☆M)",PHOENIX,Arcade
...,...,...,...
544,PRiMA MATERiA,PHOENIX,Arcade
545,Solfeggietto,PHOENIX,Arcade
546,Destr0yer,PHOENIX,Arcade
547,Lucid Dream,PHOENIX,Arcade


In [39]:
def get_full_song_name(songname):
    return songname[:-14]

def get_short_cut_name(songname):
    return songname[:-14]
remix = []
for item in songList_df_new2['song']:
    if is_short_cut(item):
        #print("Short Cut:", item)
        newname = get_short_cut_name(item)
        ismatched = False
        for j in range(len(arcade_metadata)):
            if newname == arcade_metadata[j]['song']:
                #print("is matched")
                ismatched = True
        if not ismatched:
            #print("is not matched")
            continue
    elif is_full_song(item):
        print("Full Song:", item)
        newname = get_full_song_name(item)
        ismatched = False
        for j in range(len(arcade_metadata)):
            if newname == arcade_metadata[j]['song']:
                print("is matched")
                ismatched = True
        if not ismatched:
            print("is not matched")
    else:
        isRemix = True
        for j in range(len(arcade_metadata)):
            if item == arcade_metadata[j]['song']:
                #print("Arcade: ", item)
                isRemix = False
        if isRemix:
            remix.append(item)
            #print("Remix: ", item)





Full Song: Gargoyle - FULL SONG -
is matched
Full Song: 8 6 - FULL SONG -
is matched
Full Song: Bad Apple!! feat. Nomico - FULL SONG -
is matched
Full Song: Baroque Virus - FULL SONG -
is matched
Full Song: Allegro Con Fuoco - FULL SONG -
is matched
Full Song: GOOD NIGHT - FULL SONG -
is matched
Full Song: Papasito (feat. KuTiNA) - FULL SONG -
is matched
Full Song: Full Moon - FULL SONG -
is matched
Full Song: Canon D - FULL SONG -
is matched
Full Song: Love is a Danger Zone 2 - FULL SONG -
is not matched
Full Song: Chase Me - FULL SONG -
is matched
Full Song: Dignity - FULL SONG -
is matched
Full Song: Interference - FULL SONG -
is matched
Full Song: Move That Body! - FULL SONG -
is matched
Full Song: Beat of The War 2 - FULL SONG -
is matched
Full Song: HUSH - FULL SONG -
is matched
Full Song: Chopsticks Challenge - FULL SONG -
is matched
